In [16]:
# Run and test several type4s of MCP

In [18]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio

import os

from IPython.display import Markdown, display
from datetime import datetime

load_dotenv()

True

### Type A: 100% local MCP type

In [ ]:
# test code on terminal: 
#      LIBSQL_URL="file:./memory/ed.db" npx -y mcp-memory-libsql
os.makedirs("memory", exist_ok=True)
params = {
    "command": "npx",
    "args": ["-y", "mcp-memory-libsql"],
    "env": {
        **os.environ,  # REQUIRED
        "LIBSQL_URL": "file:./memory/ed.db",
    },
}

# test the mcp server by first list the tools
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='create_entities', title='Create new entities with observations', description='Create new entities with observations', inputSchema={'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'entityType': {'type': 'string'}, 'observations': {'type': 'array', 'items': {'type': 'string'}}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities'], '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, icons=None, annotations=None, meta=None, execution=None),
 Tool(name='search_nodes', title='Search for entities and their relations using text search with relevance ranking', description='Search for entities and their relations using text search with relevance ranking', inputSchema={'type': 'object', 'properties': {'query': {'type': 'string'}, 'limit': {'type': 'number'}}, 'required': ['query'], '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=No

In [10]:
# then proceed with implementing the agent

instructions = "You use your entity tools as a persistent memory to store and recall information about your conversation"
request = "My name is Ed, I am an LLM engineer teaching courses about `AI agent`, including MCP protocol." \
"MCP is a protocol to connect agents with tools, resources, and prompt templates, making it easier to integrate agents with capabilities"
model="gpt-4o-mini"

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("udemy-mcplocal-graphmemory-conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

I've noted your details:

- **Name:** Ed
  - Role: LLM engineer
  - Focus: Teaching courses about AI agents, including the MCP protocol.

- **Concepts:**
  - **AI Agent:** Connects with tools, resources, and prompt templates to enhance capabilities.
  - **MCP Protocol:** Facilitates the integration of agents with various resources and tools.

If you need to add or modify anything, just let me know!

In [11]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("udemy-mcplocal-graphmemory-conversation"):
        result = await Runner.run(agent, "My name is Ed, what do you know about me?")
    display(Markdown(result.final_output))

I know that your name is Ed, and you are an LLM engineer, teaching courses about AI agents. You're also an expert in the MCP protocol. If there's anything specific you'd like to share or discuss, feel free!

# Test type B MCP: local with API access


In [ ]:
# test code on terminal: 
#    npm install -g dotenv-cli #one time installation
#    dotenv -e .env -- npx -y @modelcontextprotocol/server-brave-search

env = {
        **os.environ,  # REQUIRED
        "BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")
    }
params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": env}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='brave_web_search', title=None, description='Performs a web search using the Brave Search API, ideal for general queries, news, articles, and online content. Use this for broad information gathering, recent events, or when you need diverse web sources. Supports pagination, content filtering, and freshness controls. Maximum 20 results per request, with offset for pagination. ', inputSchema={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Search query (max 400 chars, 50 words)'}, 'count': {'type': 'number', 'description': 'Number of results (1-20, default 10)', 'default': 10}, 'offset': {'type': 'number', 'description': 'Pagination offset (max 9, default 0)', 'default': 0}}, 'required': ['query']}, outputSchema=None, icons=None, annotations=None, meta=None, execution=None),
 Tool(name='brave_local_search', title=None, description="Searches for local businesses and places using Brave's Local Search API. Best for queries related to physical locations

In [20]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("udemy-mcplocal-graphmemory-conversation"):
        result = await Runner.run(agent, "Get me a popular event schedule from Dec 19 to Dec 28 2025")
    display(Markdown(result.final_output))

Here's a schedule of popular events happening from December 19 to December 28, 2025:

### **Seattle, WA**
- **Pigeon Gets a Big Time Holiday Extravaganza**
  - **Dates**: December 19-21
  - **Venue**: Seattle Children’s Theatre
  - **Description**: A goofy “musical party.”

### **Washington, DC**
- **The Nutcracker by The Washington Ballet**
  - **Dates**: Through December 28
  - **Description**: A classic Christmas ballet with a special DC twist.

### **Houston, TX**
- **Elf In Concert**
  - **Dates**: December 19-21
  - **Venue**: Jones Hall for the Performing Arts
  - **Description**: The beloved film accompanied by a live orchestra.

### **San Francisco, CA**
- **Moulin Rouge! The Musical**
  - **Dates**: December 16-28
  - **Venue**: Orpheum Theatre
  - **Description**: A spectacular musical experience.

- **Home Alone with Live Orchestra**
  - **Dates**: December 19-20
  - **Venue**: San Francisco Symphony
  - **Description**: Enjoy the holiday classic film with live music.

### **India**
- **Live Performances by Popular Artists**
  - **Bengaluru**: December 19
  - **Kolkata**: December 21
  - **Mumbai**: December 26
  - **Jaipur**: December 28

### **For More Information**
- **Seattle Events**: [Seattle Events](https://www.events12.com/seattle/)
- **Washington, DC Events**: [DC Events](https://washington.org/things-to-do-this-december-in-washington-dc)
- **Houston Events**: [Houston Events](https://www.houstonmagazine.com/top-20-events-in-december/)
- **San Francisco Events**: [San Francisco Events](https://www.hautelivingsf.com/2025/11/02/events-calendar-december-2025/)

These events provide a mix of performances, festivals, and concerts across various cities.